In [0]:
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
# Estimateur 
from pyspark.ml.classification import LogisticRegression

# Load Our Transformer & Extractor pkg 
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer,IDF
from pyspark.ml.feature import StringIndexer

In [0]:
#df=spark.read.format("delta").load("/tmp/delta/Silver_sentiment")
#df.columns
def read_anc_table(name):
  df_anc=spark.read.option("multiline","true").option("header","true").csv("dbfs:/FileStore/tables/"+name+".csv")
  return df_anc
df= read_anc_table("silver_Table")

In [0]:
df=df.select( 'CommentAferPreproc','sentiment')

In [0]:
#count value 
df.groupBy('sentiment').count().show()

+---------+-----+
sentiment|count|
+---------+-----+
 Positive| 387|
 Neutral| 81|
 Negative| 110|
+---------+-----+

In [0]:
#value to pandas 
df.toPandas()["sentiment"].value_counts()

Out[8]: Positive 387
Negative 110
Neutral 81
Name: sentiment, dtype: int64

In [0]:
df= df= read_anc_table("silver_Table")
df=df.select("*").toPandas()

In [0]:
df["CommentAferPreproc"]= df["CommentAferPreproc"].str.lower()
df["CommentAferPreproc"].fillna("",inplace=True)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
cv=CountVectorizer(max_df=0.9, min_df=2)
dtm=cv.fit_transform(df["CommentAferPreproc"])
#dtm: 578 nombres d'article et 6842 les nombres des termes 
LDA=LatentDirichletAllocation(n_components=7,random_state=42)
LDA.fit(dtm)

Out[11]: LatentDirichletAllocation(n_components=7, random_state=42)

In [0]:
single_topic=LDA.components_[0]
single_topic.argsort( )# get the location for a heighst value
# ARGSORT ==> INDEX POSITIONS SORTED FROM LEAST  TO GREATEST
# top 10 values (10 greatest values)
# last 10 values of argsort()
single_topic.argsort()[-10:]
top_ten_words= single_topic.argsort()[-10:]


In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
for index in top_ten_words:
  print(cv.get_feature_names()[index])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Unzipping corpora/stopwords.zip.
faire
bien
personnel
horaires
service
plus
qu
bureau
colis
poste

In [0]:
#grab the highest propability word per topic 

for i,topic in enumerate(LDA.components_):
  print(f"les meilleures 15 sujets #{i}")
  print([cv.get_feature_names()[index] for index in topic.argsort()[-10:]])
  print('\n')
  print('\n')

les meilleures 15 sujets #0
['faire', 'bien', 'personnel', 'horaires', 'service', 'plus', 'qu', 'bureau', 'colis', 'poste']




les meilleures 15 sujets #1
['bien', 'agréable', 'super', 'service', 'colis', 'qualité', 'bonne', 'accueil', 'bon', 'produits']




les meilleures 15 sujets #2
['avis', 'to', 'personnel', 'and', 'traduit', 'origine', 'google', 'bureau', 'the', 'poste']




les meilleures 15 sujets #3
['bien', 'fermé', 'personnel', 'plus', 'service', 'bureau', 'qu', 'horaires', 'colis', 'poste']




les meilleures 15 sujets #4
['salle', 'agréable', 'pratique', 'prix', 'bon', 'accueil', 'plus', 'personnel', 'magasin', 'bien']




les meilleures 15 sujets #5
['traduit', 'avis', 'souriant', 'café', 'agréable', 'bien', 'super', 'sympa', 'bon', 'accueil']




les meilleures 15 sujets #6
['bien', 'avis', 'faire', 'bureau', 'service', 'plus', 'personnel', 'qu', 'colis', 'poste']

In [0]:
topic_results=LDA.transform(dtm)
topic_results.argmax(axis=1)
df["Topic"]=topic_results.argmax(axis=1)
mytopic_dict={0:"horaires",1:"personnel",2:"personnel",3:"service",4:"accueil",5:"accueil",6:"accueil",7:"colis"}
df["Topic_label"]=df["Topic"].map(mytopic_dict)

drop table Gold_sentiment

In [0]:
df=df[['site_id',  'address_1', 'address_2', 'address_3', 'city',
       'zip_code', 'country', 'latitude', 'longitude', 'reviewer_name',
       'review_rating', 'review_comment', 
       'dates', 'CommentAferPreproc', 'sentiment',
       'number_reviews',  'Topic_label']]

In [0]:
from pyspark.sql import SparkSession
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(df) 
sparkDF.printSchema()
sparkDF.show()

root
-- site_id: string (nullable = true)
-- address_1: string (nullable = true)
-- address_2: string (nullable = true)
-- address_3: string (nullable = true)
-- city: string (nullable = true)
-- zip_code: string (nullable = true)
-- country: string (nullable = true)
-- latitude: string (nullable = true)
-- longitude: string (nullable = true)
-- reviewer_name: string (nullable = true)
-- review_rating: string (nullable = true)
-- review_comment: string (nullable = true)
-- dates: string (nullable = true)
-- CommentAferPreproc: string (nullable = true)
-- sentiment: string (nullable = true)
-- number_reviews: string (nullable = true)
-- Topic_label: string (nullable = true)

+-------+--------------------+--------------------+---------+--------------------+--------+-------+---------+------------+--------------------+--------------------+--------------------+----------+--------------------+---------+--------------+-----------+
site_id| address_1| address_2|address_3| city|zip_code|country| latitude| longitude| reviewer_name| review_rating| review_comment| dates| CommentAferPreproc|sentiment|number_reviews|Topic_label|
+-------+--------------------+--------------------+---------+--------------------+--------+-------+---------+------------+--------------------+--------------------+--------------------+----------+--------------------+---------+--------------+-----------+
 13325A| 3 PLACE LAGELEE| null| null| RONCHAMP| 70250| FRANCE|47.699938| 6.631283|[cyrille Demesy, ...|[5,0, 5,0, 1,0, 1...|[très satisfaite ...|2021-12-18|satisfaite servic...| Positive| 11| accueil|
 13283A| PLACE EDMOND BOUR| null| null| GRAY| 70100| FRANCE| 47.44787| 5.589597|[Marie Marie, Aka...|[1,0, 1,0, 4,0, 1...|[qualité du serv...|2021-10-19|qualité service d...| Positive| 8| service|
 12957A|16 RUE JULES CHAUSSE| null| null| CHONOST| 69630| FRANCE|45.708253| 4.743598|[Carole S., Lara ...|[5,0, 5,0, 5,0, 1...|[agréable surpris...|2021-12-20|agréable surprise...| Positive| 53| horaires|
 13034A| 6 RUE VOLTAIRE| null| null| PIERRE BENITE| 69310| FRANCE|45.703494| 4.821958|[Kahena Khennou, ...|[1,0, 1,0, 1,0, 1...|[personnel en sou...|2021-01-17|personnel effecti...| Negative| 50| accueil|
 13064A|67 AVENUE JEAN JA...| null| null| ST FONS| 69190| FRANCE|45.710402| 4.855148|[Ebru. ___, Tiffa...|[1,0, 1,0, 1,0, 1...|[tout est lamenta...|2022-01-14|lamentable servic...| Positive| 60| accueil|
 13127A| 3 MONTEE DU CHATEAU| null| null| ST BONNET DE MURE| 69720| FRANCE|45.692369| 5.029318|[Pascale Thevenet...|[1,0, 1,0, 1,0, 5...|[je constate qu i...|2021-12-18|constate qu aucun...| Negative| 35| horaires|
 12971A| 24 COURS GAMBETTA| null| null| LYON GABRIEL PERI| 69007| FRANCE|45.754893| 4.843721|[alice martin, Fl...|[5,0, 1,0, 5,0, 1...|[personnel très a...|2021-04-22|personnel agréabl...| Positive| 25| horaires|
 13128A|4 RUE DU CENTRE B...| null| null| ST LAURENT DE MURE| 69720| FRANCE|45.687183| 5.048422|[Sylvie Grosjean,...|[1,0, 1,0, 2,0, 1...|[poste flambant n...|2021-12-27|poste flambant ne...| Positive| 74| horaires|
 13115A| PLACE DU COSTEL| null| null| CORBAS| 69960| FRANCE|45.671591| 4.898851|[R T, Nizar Ayari...|[4,0, 5,0, 1,0, 1...|[ouvert du lundi ...|2021-11-18|lundi samedi appr...| Positive| 55| accueil|
 13059A|10 RUE JEAN ET CA...| null| null| ST CYR AU MONT D OR| 69450| FRANCE|45.816598| 4.817276|[Anais Nana, Jacq...|[1,0, 2,0, 1,0, 2...|[déjà que les fac...|2021-11-18|facteurs incapabl...| Positive| 43| service|
 13126A|16 RUE JACQUES PR...| null| null| RILLIEUX LA PE PAL| 69140| FRANCE|45.816793| 4.90199|[Sona N, Moi, sen...|[5,0, 1,0, 1,0, 1...|[je comprends pas...|2022-01-11|comprends comment...| Negative| 94| accueil|
 12990A|3 CHEMIN DE CHANT...| null| null| FRANCHEVILLE| 69340| FRANCE|45.738391| 4.774365|[Jordan Ferreira,...|[1,0, 1,0, 1,0, 1...|[dommage qu'on pu...|2021-10-19|dommage qu puisse...| Negative| 83| accueil|
 12984A| RUE DES ROCHES|AU PETIT CREUX DU...| null| ECHALAS| 69700| FRANCE|45.551309| 4.715414|[Christine Vallet...|[5

In [0]:
dbutils.fs.rm("dbfs:/FileStore/tables/Golde_Table.csv", True)

Out[18]: True

In [0]:
sparkDF.write.format("csv").option("header", "true").save("dbfs:/FileStore/tables/Golde_Table.csv")

In [0]:
display(sparkDF)

site_id address_1 address_2 address_3 city zip_code country latitude longitude reviewer_name review_rating review_comment dates CommentAferPreproc sentiment number_reviews Topic_label 13325A 3 PLACE LAGELEE null null RONCHAMP 70250 FRANCE 47.699938 6.631283 [cyrille Demesy, Christophe (“Sirozy”), Emma De Laurent, ANNE N, Lamouche Zobi] [5,0, 5,0, 1,0, 1,0, 3,0] [très satisfaite des services et de l'accueil, beau coup de bons conseils et d'aide aux personnes...bravo !jocelyne j de ronchamp, personnel agréable, je n'ai pas la possibilité de mettre moins qu'une étoile, quel dommage. accueil plus que médiocre depuis bien longtemps déjà ...la personne qui s'occupe de l'accueil n'a pas même daigné passer un coup de téléphone dans une poste voisine pour ensuite me traiter à demi-mots de menteuse et de voleuse !, aucune confidentialité ! accueille médiocre..., (traduit par google) correct(avis d'origine)correct] 2021-12-18 satisfaite services accueil beau coup bons conseils aide personnes bravo jocelyne ronchamp personnel agréable possibilité mettre qu étoile dommage accueil plus médiocre bien personne occupe accueil daigné passer coup téléphone poste voisine ensuite traiter demi mots menteuse voleuse aucune confidentialité accueille médiocre traduit google correct avis origine correct Positive 11 accueil 13283A PLACE EDMOND BOUR null null GRAY 70100 FRANCE 47.44787 5.589597 [Marie Marie, Akadream, Ophélie Marmier, vanessa blanche, Clément Cotteret, Axel Dumais, Arnaud Ortega, VI Vie, Biot Mickael] [1,0, 1,0, 4,0, 1,0, 5,0, 3,0, 1,0, 5,0, 1,0] [qualité du service déplorable .il faut en attendre le minimum ., facteur qui ne se présente pas au domicile, qui possède mon numéro de téléphone mais qui laisse quand même un avis de passage alors que c'est le confinement, que je ne bouge pas de chez moi et que je suis levé depuis 7h30 le matin.c'est tout bonnement de l'incompétence, cette personne n'a simplement pas fait son boulot et cela devrait être considéré comme étant une faute professionnelle au point d'être licencié.pourquoi laisser cette incompétence alors que des personnes sans emploi et pleins de bonnes volontés souhaitent trouver du travail ?enfin bref, j'ai jamais eu une si mauvaise expérience dans la livraison qu'aujourd'hui, je ne recommande absolument pas cette poste, ils ne méritent pas qu'on se tourne vers eux., personnel parfois (rarement) un peu froid mais qui reste professionnel et qui prend le temps d'expliquer correctement lorsqu'on a besoin d'aide, les prix de la poste restent très élevés je ne sais pas si ce sont les même tarifs partout (j'imagine et j'espère que oui) mais l'envois des colis coûte une forture, personnel incompétent qui ne veut rien faire surtout un homme moustachu. bureau de poste à éviter absolument car ne serve à rien ., personnels accueillants et professionnels, , , , ] 2021-10-19 qualité service déplorable faut attendre minimum facteur présente domicile possède numéro téléphone laisse avis passage confinement bouge levé 7h30 matin bonnement incompétence personne simplement boulot devrait considéré faute professionnelle point licencié incompétence personnes emploi pleins bonnes volontés souhaitent trouver travail bref jamais mauvaise expérience livraison qu aujourd recommande absolument poste méritent qu tourne personnel rarement froid professionnel prend temps expliquer correctement lorsqu besoin aide prix poste élevés sais tarifs partout imagine espère oui envois colis coûte forture personnel incompétent veut rien faire homme moustachu bureau poste éviter absolument serve rien personnels accueillants professionnels Positive 8 service 12957A 16 RUE JULES CHAUSSE null null CHONOST 69630 FRANCE 45.708253 4.743598 [Carole S., Lara Debard, Aurèle FERRAT, Kio Kiowa, Mathieu Verrier, George Leroix, Laurine Grillo-farissier, Rajaa El kbiri, Robin Lainé, SPEEPING, Blb 94, Brig Dain, Marion D, Romain SALIS, Francesca Chilla, Amelie Prudhon, Laurene Tignat, bozon thomas, Manuel marques neves, Titouan Richard

In [0]:
def sub_unmount(str_path):
    if any(mount.mountPoint == str_path for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(str_path)

sub_unmount('/mnt/DonneeGold')

/mnt/DonneeGold has been unmounted.

In [0]:
dbutils.fs.mount(
  source = "wasbs://scrapdata@storagedataplatform2.blob.core.windows.net",
  mount_point = "/mnt/DonneeGold",
  extra_configs = {"fs.azure.account.key.storagedataplatform2.blob.core.windows.net":"Mby7ZbUsaA8kojqcnOZoCw53gmbd2y7mPPmHyu+XLi5/ZINtxHjclljc5Q8p8NDVibx9fpx06ngV+ASttMSaFw=="})

Out[22]: True

In [0]:
def from_pandas_toSpark(df):
  #Create PySpark DataFrame from Pandas
  sparkDF=spark.createDataFrame(df) 
  sparkDF.printSchema()
  sparkDF.show()
  return sparkDF 
#df=from_pandas_toSpark(df)

root
-- site_id: string (nullable = true)
-- address_1: string (nullable = true)
-- address_2: string (nullable = true)
-- address_3: string (nullable = true)
-- city: string (nullable = true)
-- zip_code: string (nullable = true)
-- country: string (nullable = true)
-- latitude: string (nullable = true)
-- longitude: string (nullable = true)
-- reviewer_name: string (nullable = true)
-- review_rating: string (nullable = true)
-- review_comment: string (nullable = true)
-- dates: string (nullable = true)
-- CommentAferPreproc: string (nullable = true)
-- sentiment: string (nullable = true)
-- number_reviews: string (nullable = true)
-- Topic_label: string (nullable = true)

+-------+--------------------+--------------------+---------+--------------------+--------+-------+---------+------------+--------------------+--------------------+--------------------+----------+--------------------+---------+--------------+-----------+
site_id| address_1| address_2|address_3| city|zip_code|country| latitude| longitude| reviewer_name| review_rating| review_comment| dates| CommentAferPreproc|sentiment|number_reviews|Topic_label|
+-------+--------------------+--------------------+---------+--------------------+--------+-------+---------+------------+--------------------+--------------------+--------------------+----------+--------------------+---------+--------------+-----------+
 13325A| 3 PLACE LAGELEE| null| null| RONCHAMP| 70250| FRANCE|47.699938| 6.631283|[cyrille Demesy, ...|[5,0, 5,0, 1,0, 1...|[très satisfaite ...|2021-12-18|satisfaite servic...| Positive| 11| accueil|
 13283A| PLACE EDMOND BOUR| null| null| GRAY| 70100| FRANCE| 47.44787| 5.589597|[Marie Marie, Aka...|[1,0, 1,0, 4,0, 1...|[qualité du serv...|2021-10-19|qualité service d...| Positive| 8| service|
 12957A|16 RUE JULES CHAUSSE| null| null| CHONOST| 69630| FRANCE|45.708253| 4.743598|[Carole S., Lara ...|[5,0, 5,0, 5,0, 1...|[agréable surpris...|2021-12-20|agréable surprise...| Positive| 53| horaires|
 13034A| 6 RUE VOLTAIRE| null| null| PIERRE BENITE| 69310| FRANCE|45.703494| 4.821958|[Kahena Khennou, ...|[1,0, 1,0, 1,0, 1...|[personnel en sou...|2021-01-17|personnel effecti...| Negative| 50| accueil|
 13064A|67 AVENUE JEAN JA...| null| null| ST FONS| 69190| FRANCE|45.710402| 4.855148|[Ebru. ___, Tiffa...|[1,0, 1,0, 1,0, 1...|[tout est lamenta...|2022-01-14|lamentable servic...| Positive| 60| accueil|
 13127A| 3 MONTEE DU CHATEAU| null| null| ST BONNET DE MURE| 69720| FRANCE|45.692369| 5.029318|[Pascale Thevenet...|[1,0, 1,0, 1,0, 5...|[je constate qu i...|2021-12-18|constate qu aucun...| Negative| 35| horaires|
 12971A| 24 COURS GAMBETTA| null| null| LYON GABRIEL PERI| 69007| FRANCE|45.754893| 4.843721|[alice martin, Fl...|[5,0, 1,0, 5,0, 1...|[personnel très a...|2021-04-22|personnel agréabl...| Positive| 25| horaires|
 13128A|4 RUE DU CENTRE B...| null| null| ST LAURENT DE MURE| 69720| FRANCE|45.687183| 5.048422|[Sylvie Grosjean,...|[1,0, 1,0, 2,0, 1...|[poste flambant n...|2021-12-27|poste flambant ne...| Positive| 74| horaires|
 13115A| PLACE DU COSTEL| null| null| CORBAS| 69960| FRANCE|45.671591| 4.898851|[R T, Nizar Ayari...|[4,0, 5,0, 1,0, 1...|[ouvert du lundi ...|2021-11-18|lundi samedi appr...| Positive| 55| accueil|
 13059A|10 RUE JEAN ET CA...| null| null| ST CYR AU MONT D OR| 69450| FRANCE|45.816598| 4.817276|[Anais Nana, Jacq...|[1,0, 2,0, 1,0, 2...|[déjà que les fac...|2021-11-18|facteurs incapabl...| Positive| 43| service|
 13126A|16 RUE JACQUES PR...| null| null| RILLIEUX LA PE PAL| 69140| FRANCE|45.816793| 4.90199|[Sona N, Moi, sen...|[5,0, 1,0, 1,0, 1...|[je comprends pas...|2022-01-11|comprends comment...| Negative| 94| accueil|
 12990A|3 CHEMIN DE CHANT...| null| null| FRANCHEVILLE| 69340| FRANCE|45.738391| 4.774365|[Jordan Ferreira,...|[1,0, 1,0, 1,0, 1...|[dommage qu'on pu...|2021-10-19|dommage qu puisse...| Negative| 83| accueil|
 12984A| RUE DES ROCHES|AU PETIT CREUX DU...| null| ECHALAS| 69700| FRANCE|45.551309| 4.715414|[Christine Vallet...|[5

In [0]:
df.write.option("header",True).format("com.databricks.spark.csv").save("/mnt/DonneeGold/Gold_Table") 

In [0]:
dbutils.fs.unmount("/mnt/DonneeGold") 

/mnt/DonneeGold has been unmounted.
Out[31]: True

In [0]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
connect_str = 'DefaultEndpointsProtocol=https;AccountName=storagedataplatform2;AccountKey=Mby7ZbUsaA8kojqcnOZoCw53gmbd2y7mPPmHyu+XLi5/ZINtxHjclljc5Q8p8NDVibx9fpx06ngV+ASttMSaFw==;EndpointSuffix=core.windows.net'
client=BlobClient.from_connection_string(connect_str,container_name="data",blob_name="Gold_Table.csv")
with open("/dbfs/FileStore/tables/Gold_Table.csv","rb")as f:
  client.upload_blob(f)
  

In [0]:
to_bob_storage("Gold_Table")